In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from modules import *
from astropy.table import Table
import scipy.signal as signal
import scipy.optimize as optimize
from scipy.signal import savgol_filter

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 61285257'
lcs = get_lightcurves(TIC, author = 'SPOC', cadence='short', use_till=1)

In [ ]:
new_lcs = []

for lc in lcs:
    df = pd.DataFrame({'time': lc.time.jd, 'flux': np.array(lc.flux, dtype='f')}).dropna(inplace=False)
    new_lc = lk.LightCurve(time= df['time'], flux= df['flux'])
    new_lc = apply_savgol_filter(new_lc.time.jd, new_lc.flux, want = 'lc', displaygraphs= False, window_length_for_remove=1500)
    new_lc = straight_lines(new_lc, cadence_magnifier= 20)
    new_lcs.append(new_lc)

lightcurve = combine_lightcurves(new_lcs)
lightcurve.plot()

In [ ]:
periodogram = lightcurve.to_periodogram(maximum_frequency = 50)
periodogram.plot()

In [ ]:
period = 1 / 4.264051779
t0 = 1900.09422143

lc_phased = lightcurve.fold(period=period, epoch_time=t0)

lc_phased_binned = lc_phased.bin(1/24/60)

fig, ax = plt.subplots(figsize = (8,5))

lc_phased.plot(ax = ax, marker = '.', linewidth = 0, color = 'orange', alpha = 0.2, markersize = 3, label = 'unbinned')
lc_phased_binned.plot(ax = ax, marker = '.', linewidth = 0, color = 'k', alpha = 0.8, markersize = 6, label = 'binned')

ax.legend()
ax.set_xlabel('Phase')
ax.set_ylabel('Flux')
ax.set_title('Phased Light Curve')
plt.show()

In [ ]:
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

window_length = 51 
polyorder = 3     

lc_binned_smoothed = savgol_filter(lc_phased_binned.flux.value, window_length, polyorder)

fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(lc_phased_binned.time.value, lc_binned_smoothed, color='k', linewidth=2, label='smoothed')
ax.set_xlabel('Phase')
ax.set_ylabel('Flux')

